*March 2018*

**Introduction**

   Problem case:

TL;DR: In a nutshell, create an algorithm to automate nucleus detection in divergent images  to advance medical discovery and unlock faster cures. [(Read more here)](https://datasciencebowl.com)


   Solution:

The solution is to build a CNN in the shape of a U, known as a U-Net. The U-Net, according to the [authors at the University of Freiburg](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/), is best suited for convolutional neural networks for Biomedical Image Segmentation.

The CNN will be built on the training data and applied to the test data.

**U-Net architecture flow:**

![U-Net architecture flow:](https://lmb.informatik.uni-freiburg.de/Publications/2015/RFB15a/u-net-architecture.png)

The main idea of this 'fully connected network' is to supplement a usual contracting network by successive layers, where pooling operators are replaced by upsampling operators. Hence, these layers (the right side of the 'U') **increase** the resolution of the output. 

*Excerpt* from [original paper:](https://arxiv.org/pdf/1505.04597.pdf) 

**Network Architecture**

The network architecture consists of a contracting path (left side) and an expansive path (right side). The contracting path follows the typical architecture of a convolutional network. It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), each followed by a rectified linear unit (ReLU) and a 2x2 max pooling operation with stride 2 for downsampling. At each downsampling step we double the number of feature channels. Every step in the expansive path consists of an upsampling of the feature map followed by a 2x2 convolution (“up-convolution”) that halves the number of feature channels, a concatenation with the correspondingly cropped feature map from the contracting path, and two 3x3 convolutions, each followed by a ReLU. The cropping is necessary due to the loss of border pixels in every convolution. At the final layer a 1x1 convolution is used to map each 64- component feature vector to the desired number of classes. In total the network has 23 convolutional layers.

*End excerpt.*



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import numpy as np


def make_df(train_path, test_path, img_size):
    train_ids = next(os.walk(train_path))[1]
    test_ids = next(os.walk(test_path))[1]
    X_train = np.zeros((len(train_ids), img_size, img_size, 3), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), img_size, img_size, 1), dtype=np.bool)
    for i, id_ in enumerate(train_ids):
        path = train_path + id_
        img = cv2.imread(path + '/images/' + id_ + '.png')
        img = cv2.resize(img, (img_size, img_size))
        X_train[i] = img
        mask = np.zeros((img_size, img_size, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = cv2.imread(path + '/masks/' + mask_file, 0)
            mask_ = cv2.resize(mask_, (img_size, img_size))
            mask_ = mask_[:, :, np.newaxis]
            mask = np.maximum(mask, mask_)
        Y_train[i] = mask
    X_test = np.zeros((len(test_ids), img_size, img_size, 3), dtype=np.uint8)
    sizes_test = []
    for i, id_ in enumerate(test_ids):
        path = test_path + id_
        img = cv2.imread(path + '/images/' + id_ + '.png')
        sizes_test.append([img.shape[0], img.shape[1]])
        img = cv2.resize(img, (img_size, img_size))
        X_test[i] = img

    return X_train, Y_train, X_test, sizes_test

Define the U-Net model

In this next section, we define the U-Net model, 9 parts in all. Each part in the architecture consists of 4 operations:

    1. Convolve
    2. Dropout
    3. Convolve
    4. Maxpooling
 
 If you do not what there are about please check out Stanfords fabled [Deep Learning specialization](https://www.coursera.org/specializations/deep-learning) on Coursera, run by professor Andrew Ng, the unicorn of AI. 
 Now when it comes to the activation function, there are a whole host that you can test for increased accuracy, like elu, relu, selu and tanh.

In [ ]:
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda


def Unet(img_size):
    inputs = Input((img_size, img_size, 3))
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    print('c1')
    c2 = Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    print('c2')
    c3 = Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    print('c3')
    c4 = Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    print('c4')
    c5 = Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c5)
    print('c5')
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c6)
    print('c6')
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c7)
    print('c7')
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c8)
    print('c8')
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='valid')(c9)
    print('c9')
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    return model

Define your generator

In this section we are going to generate images by performing some operations on the images, like horizontally flipping them, and vertically too.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


def generator(xtr, xval, ytr, yval, batch_size):
    data_gen_args = dict(horizontal_flip=True,
                         vertical_flip=True,
                         rotation_range=90.,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.1)
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    image_datagen.fit(xtr, seed=10)
    mask_datagen.fit(ytr, seed=10)
    image_generator = image_datagen.flow(xtr, batch_size=batch_size, seed=10)
    mask_generator = mask_datagen.flow(ytr, batch_size=batch_size, seed=10)
    train_generator = zip(image_generator, mask_generator)

    val_gen_args = dict()
    image_datagen_val = ImageDataGenerator(**val_gen_args)
    mask_datagen_val = ImageDataGenerator(**val_gen_args)
    image_datagen_val.fit(xval, seed=10)
    mask_datagen_val.fit(yval, seed=10)
    image_generator_val = image_datagen_val.flow(xval, batch_size=batch_size, seed=10)
    mask_generator_val = mask_datagen_val.flow(yval, batch_size=batch_size, seed=10)
    val_generator = zip(image_generator_val, mask_generator_val)

    return train_generator, val_generator

The IoU metric is an evaluation metric that tells how well your detected bounding boxes overlap the actual object on your image. Here is the basic intuition of it:
![IoU](https://i.stack.imgur.com/JlHnn.jpg)

So later when the model is compiled, this evaluation metric will be used to determine the best model to be compiled according to the model training data. For more on this metric and it's use, check out this [paper on Fully Convolutional Networks for Semantic Segmentation.](https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf) 

A good loss funtion is based on the dice coefficient. Here is an excerpt from a [really good paper](https://arxiv.org/pdf/1711.03954.pdf) that will give you a good intuition about the dice coefficient and how it is used in a loss funtion:

*begin excerpt:*

The dice coefficient is a popular and largely used cost function in segmentation problems. Considering the predicted region P and the groundtruth region G, and by denoting |P| and |G| the sum of elements in each area, the dice coefficient is twice the ratio of the intersection over the sum of areas:
DiceCoef(P, G) = 2|P ∩ G| / ( |P| + |G| )
A perfect segmentation result is given by a dice coefficient of 1, while a dice coefficient of 0 refers to a completely mistaken segmentation.

*end excerpt.*

Another loss function is based on cross entropy. Cross-entropy loss, or log loss, measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label. Read more about it [here.](http://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html)

I am going to use the dice coefficient in a custom loss function that uses binary cross entropy too. It is aptly called the bce_dice_loss.



In [ ]:
import tensorflow as tf
import numpy as np
from keras import backend as K
from keras.losses import binary_crossentropy

# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2, y_true)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    return 0.5 * binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

RLE encoding for submission.

In [ ]:
from skimage.morphology import label

def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

Now we are ready to run our model on the training data and apply it to our test data. 
For my optimizer I am going to test using Adam, which converges *faster* than Stochastic Gradient Descent(SGD) but can generalize [*worse*.](https://arxiv.org/pdf/1705.08292.pdf)
I'll also test SGD. Hopefully it finishes running before hardware allowance cutoff here on kaggle!

In [ ]:
import cv2
import os
import pandas as pd
from sklearn.model_selection import train_test_split


if __name__ == "__main__":
    img_size = 256
    batch_size = 32
    train_path = '../input/stage1_train/'
    test_path = '../input/stage2_test_final/'
    
    X_train, Y_train, X_test, sizes_test = make_df(train_path, test_path, img_size)
    xtr, xval, ytr, yval = train_test_split(X_train, Y_train, test_size=0.1, random_state=7)
    train_generator, val_generator = generator(xtr, xval, ytr, yval, batch_size)
    print('imported')
    print(xtr)
    model = Unet(img_size)
    print('Model summary:')
    model.summary()
    #model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[mean_iou])
    model.compile(optimizer='sgd', loss=bce_dice_loss, metrics=[mean_iou])
    print('going to model fit generator now')
    #model.fit_generator(train_generator, steps_per_epoch=len(xtr)/6, epochs=15,
    model.fit_generator(train_generator, steps_per_epoch=25, epochs=,
                        validation_data=val_generator, validation_steps=len(xval)/batch_size)
    
    preds_test = model.predict(X_test, verbose=1)

    preds_test_upsampled = []
    for i in range(len(preds_test)):
        preds_test_upsampled.append(cv2.resize(preds_test[i], 
                                           (sizes_test[i][1], sizes_test[i][0])))
        
    test_ids = next(os.walk(test_path))[1]
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ids):
        rle = list(prob_to_rles(preds_test_upsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
        
        
    sub = pd.DataFrame()
    sub['ImageId'] = new_test_ids
    sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
    sub.to_csv('submission_tanh.csv', index=False)

OK so thats about that, feel free to fork the notebook, play around with the data genarator parameters, and also the loss function, also the hyper parameters like epochs etc...
Happy training! and if you liked the notebook please UPVOTE! :-)